In [21]:
from importlib import reload
import model
import torch
reload(model)

model = model.GPT2("./config.json")

Initializing new GPT-2...


In [22]:
p = 0.8
context_length = 10
B = 3
attn_mask = torch.tril(torch.ones(context_length, context_length)).to(torch.bool)
attn_unmask = torch.rand(size=(context_length, context_length)) <= p
ans = attn_mask | attn_unmask

sentence = torch.randint(0, 10, size = (B, context_length + 1))
input_ids = sentence[:, :-1]
targets = sentence[:, 1:]
logits = model.training_step((input_ids, targets), None)

torch.Size([3, 10, 50257]) torch.Size([3, 10])


RuntimeError: GPT2 is not attached to a `Trainer`.

In [ ]:
import torch

def get_anneal_attn_mask(seq_len, bsz, dtype, device, attn_mask_ratio):
    mask = torch.full((seq_len, seq_len), 0, device=device)
    mask_cond = torch.arange(mask.size(-1), device=device)
    mask.masked_fill_(mask_cond < (mask_cond + 1).view(mask.size(-1), 1), 1)
    causal_mask = mask.to(dtype)
    
    random_mask = torch.bernoulli(torch.full((seq_len, seq_len), 0.0, device=device) + attn_mask_ratio)

    anneal_mask = torch.logical_or(causal_mask, random_mask)
    expanded_mask = anneal_mask[None, None, :, :].expand(bsz, 1, seq_len, seq_len)
    inverted_mask = 1.0 - expanded_mask.to(dtype)

    return inverted_mask.masked_fill(inverted_mask.to(torch.bool), torch.finfo(dtype).min)